In [21]:
!git clone https://github.com/nmtremblay/lofi-samples.git
!pip install music21
!pip install np_utils

fatal: destination path 'lofi-samples' already exists and is not an empty directory.
     |████████████████████████████████| 11.4MB 3.1MB/s 


In [22]:
import glob
import pickle
import numpy as np
import pandas as pd
from music21 import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
notes = []
for file in glob.glob("lofi-samples/samples/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [24]:
sequence_length = 20

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_output = []
network_input = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i+sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
n_vocab = len(set(notes))
network_input = network_input / float(n_vocab)

from keras.utils.np_utils import to_categorical
try:
    network_output = np_utils.to_categorical(network_output)
except ValueError:
    pass

In [32]:
model = Sequential()

model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.fit(network_input, network_output, epochs=20, batch_size=64)

Epoch 1/20
25/25 [==============================] - 9s 342ms/step - loss: 5.2099
Epoch 2/20
25/25 [==============================] - 9s 344ms/step - loss: 5.1114
Epoch 3/20
25/25 [==============================] - 9s 350ms/step - loss: 5.0844
Epoch 4/20
25/25 [==============================] - 8s 335ms/step - loss: 5.0697
Epoch 5/20
25/25 [==============================] - 8s 335ms/step - loss: 5.0578
Epoch 6/20
25/25 [==============================] - 8s 336ms/step - loss: 5.0317
Epoch 7/20
25/25 [==============================] - 8s 337ms/step - loss: 4.9871
Epoch 8/20
25/25 [==============================] - 8s 336ms/step - loss: 4.9550
Epoch 9/20
25/25 [==============================] - 8s 337ms/step - loss: 4.9156
Epoch 10/20
25/25 [==============================] - 8s 333ms/step - loss: 4.8612
Epoch 11/20
25/25 [==============================] - 8s 337ms/step - loss: 4.8243
Epoch 12/20
25/25 [==============================] - 8s 335ms/step - loss: 4.7662
Epoch 13/20
25/25 [======

In [68]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

for note_index in range(100): 
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)    
    prediction = model.predict(prediction_input, verbose=0)    
    index = np.argmax(prediction)
    result = int_to_note[index]

    prediction_output.append(result)    

    
    pattern.ravel()
    
    patternbeta = np.append(pattern, [index])
    
    pattern = patternbeta[1:len(patternbeta)]
    

In [63]:
offset = 0
output_notes = []

for pattern in prediction_output:
    if('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

In [64]:
us = environment.UserSettings()
us.getSettingsPath()

PosixPath('/root/.music21rc')

In [65]:
s = stream.Stream(output_notes)
mf = s.write('midi', fp="lofi-samples/testOutput.mid")